<a href="https://colab.research.google.com/github/usma11dia0/web_scraping_on_colab/blob/main/web_scraping_on_colab_triview.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium
pip install selenium

Executing: /tmp/apt-key-gpghome.UXzG64NeZZ/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: public key "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.4QWzEb5tOe/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: public key "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.TZAo3n1OA8/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: public key "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Get:1 http://deb.debian.org/debian buster InRelease [122 kB]
Get:2 http://deb.debian.org/debian bust

In [3]:
#環境変数設定
TARGET_URL = 'https://tribeau.jp/surgery_sites/1/case_reports'
#  https://tribeau.jp/surgery_sites/1/case_reports?page=2
DRIVER_WAIT_TIME = 3

# 各種ファイル保存先
SUBMIT_FILE_PATH ='/content/drive/MyDrive/Colab Notebooks/dev/web_scraping_crownstrategy/accumulated_list_tribeau_eye.xlsx'

#取得データ格納先
result_dict = {
    'クリニック名': [],
    '施術名':[],
    'ドクター名': [],
    '症例画像':[],
}

In [9]:
#標準ライブラリ
import os
import math
import json

#サードパーティライブラリ
import pandas as pd
from google.colab import drive
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from logging import (
    getLogger,
    StreamHandler,
    DEBUG,
    INFO,
    Formatter,
    config
)
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#関数定義
# def add_store_url(elements, store_urls, count_skip):
#     for a_element in a_elements:
#         store_url = a_element.get_attribute('href')
#         if store_url not in accu_set:
#             store_urls.append(store_url)
#         else:
#             count_skip += 1
#     return store_urls, count_skip

In [10]:
options = Options()
options.add_argument('--no-sandbox')
options.add_argument('--headless')
options.add_argument('--disable-gpu')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--ignore-certificate-errors')
options.add_argument("--disable-extensions")
options.add_argument("--disable-popup-blocking")
options.add_argument(
       "user-agent=Mozilla/5.0 (X11; Linux x86_64; rv:93.0) Gecko/20100101 Firefox/93.0"
    )
options.add_experimental_option("prefs", {
    "profile.managed_default_content_settings.images": 2,  # 画像の無効化
    "profile.managed_default_content_settings.plugins": 2,  # プラグインの無効化
})

driver = webdriver.Chrome(options=options)
driver.implicitly_wait(DRIVER_WAIT_TIME)

In [12]:
#カスタムロガーの設定
with open('/content/drive/MyDrive/Colab Notebooks/dev/web_scraping_crownstrategy/logging_config.json', 'r') as f:
    logger_config = json.load(f)
config.dictConfig(logger_config)
logger = getLogger('main')

try:
      driver.get(TARGET_URL)
      logger.debug(f'{TARGET_URL}：トップページへ移動しました')
except Exception as e:
      logger.error(f'{TARGET_URL}：トップページへ移動出来ませんでした: [e]')
      raise

2023-10-19 12:35:40,686 [DEBUG] main: https://tribeau.jp/surgery_sites/1/case_reports：トップページへ移動しました


In [ ]:
#店舗リストの全ページ数を導出
span_elements = driver.find_elements(By.CSS_SELECTOR,".c-page-count__num")
num_all_stores = int(span_elements[-1].text)
num_all_pages = math.ceil(num_all_stores / STORES_PER_PAGE)

#積み上げリストから既存取得の店舗URLを抽出
try:
  df_accu = pd.read_csv(ACCU_LIST_PATH, usecols=[3])
  accu_list = df_accu.iloc[:, 0].values.tolist()
  accu_set = set(accu_list)
except FileNotFoundError:
  accu_set = set()

# 目標件数を満たすまでstore_urlを取得
# 1ページ目
try:
  a_elements = driver.find_elements(By.CSS_SELECTOR,".list-rst__rst-name a")
  store_urls = []
  count_skip = 0
  store_urls, count_skip = add_store_url(a_elements, store_urls, count_skip)
  logger.debug(
    f'{TARGET}：店舗リスト取得 (取得数:{len(store_urls)}, 重複数：{count_skip})'
    )
except  Exception as e:
  logger.error(f'{TARGET}：店舗リストへ移動出来ませんでした: [e]')
  raise

#2ページ目以降
i = 1
while len(store_urls) < NUM_TO_FETCH:
  i += 1
  page_url = f'{TARGET_URL}rstLst/{i}/'
  try:
    if i < num_all_pages:
      count_skip = 0
      driver.get(page_url)
      a_elements = driver.find_elements(By.CSS_SELECTOR,".list-rst__rst-name a")
      store_urls, count_skip = add_store_url(a_elements, store_urls, count_skip)
      logger.debug(
        f'{TARGET}：店舗リスト{i}取得 (累計:{len(store_urls)}, 重複：{count_skip})'
      )
    else:
      logger.info(f'{TARGET}：取得出来る店舗情報がありません')
  except Exception as e:
    logger.error(f'{TARGET}：店舗リスト{i}へ移動出来ませんでした: [e]')

logger.info(f'{TARGET}：店舗URL取得完了:{len(store_urls)}個')


2023-04-14 21:36:44,319 [DEBUG] main: 神奈川：店舗リスト取得 (取得数:20, 重複数：0)
2023-04-14 21:36:52,352 [DEBUG] main: 神奈川：店舗リスト2取得 (累計:40, 重複：0)
2023-04-14 21:36:59,825 [DEBUG] main: 神奈川：店舗リスト3取得 (累計:60, 重複：0)
2023-04-14 21:37:06,834 [DEBUG] main: 神奈川：店舗リスト4取得 (累計:80, 重複：0)
2023-04-14 21:37:12,229 [DEBUG] main: 神奈川：店舗リスト5取得 (累計:100, 重複：0)
2023-04-14 21:37:17,621 [DEBUG] main: 神奈川：店舗リスト6取得 (累計:120, 重複：0)
2023-04-14 21:37:23,521 [DEBUG] main: 神奈川：店舗リスト7取得 (累計:140, 重複：0)
2023-04-14 21:37:30,548 [DEBUG] main: 神奈川：店舗リスト8取得 (累計:160, 重複：0)
2023-04-14 21:37:37,523 [DEBUG] main: 神奈川：店舗リスト9取得 (累計:180, 重複：0)
2023-04-14 21:37:55,514 [DEBUG] main: 神奈川：店舗リスト10取得 (累計:200, 重複：0)
2023-04-14 21:38:01,352 [DEBUG] main: 神奈川：店舗リスト11取得 (累計:220, 重複：0)
2023-04-14 21:38:06,634 [DEBUG] main: 神奈川：店舗リスト12取得 (累計:240, 重複：0)
2023-04-14 21:38:12,683 [DEBUG] main: 神奈川：店舗リスト13取得 (累計:260, 重複：0)
2023-04-14 21:38:20,130 [DEBUG] main: 神奈川：店舗リスト14取得 (累計:280, 重複：0)
2023-04-14 21:38:28,012 [DEBUG] main: 神奈川：店舗リスト15取得 (累計:300, 重複：0)
2023-04

In [ ]:
#店舗個別ページにて情報取得
page_num = 0
for store_url in store_urls:
  try:
      driver.get(store_url)
      page_num += 1
      logger.debug(f'{TARGET}：店舗ページ{page_num}へ移動しました')
  except Exception as e:
      logger.error(f'{TARGET}：店舗ページへ移動出来ませんでした: [e]')

  try:
      #店舗名/住所/電話番号取得
      store_name = driver.find_element(By.CSS_SELECTOR,".display-name").text
      phone_number = driver.find_element(By.CSS_SELECTOR,".rstinfo-table__tel-num").text
      address = driver.find_element(By.CSS_SELECTOR,".rstinfo-table__address").text
      logger.debug(f'{TARGET}：店舗ページ{page_num}の取得完了')
      #結果格納
      result_dict['店名'].append(store_name)
      result_dict['電話番号'].append(phone_number)
      result_dict['住所'].append(address)
      result_dict['URL'].append(store_url)
  except Exception as e:
      logger.error(f'{TARGET}：{store_name}の店舗情報取得に失敗しました: [e]')

df_result = pd.DataFrame(result_dict)
df_result

2023-04-14 21:42:18,545 [DEBUG] main: 神奈川：店舗ページ1へ移動しました
2023-04-14 21:42:18,813 [DEBUG] main: 神奈川：店舗ページ1の取得完了
2023-04-14 21:42:22,241 [DEBUG] main: 神奈川：店舗ページ2へ移動しました
2023-04-14 21:42:22,483 [DEBUG] main: 神奈川：店舗ページ2の取得完了
2023-04-14 21:42:25,962 [DEBUG] main: 神奈川：店舗ページ3へ移動しました
2023-04-14 21:42:26,080 [DEBUG] main: 神奈川：店舗ページ3の取得完了
2023-04-14 21:42:28,905 [DEBUG] main: 神奈川：店舗ページ4へ移動しました
2023-04-14 21:42:29,018 [DEBUG] main: 神奈川：店舗ページ4の取得完了
2023-04-14 21:42:32,194 [DEBUG] main: 神奈川：店舗ページ5へ移動しました
2023-04-14 21:42:32,321 [DEBUG] main: 神奈川：店舗ページ5の取得完了
2023-04-14 21:42:35,516 [DEBUG] main: 神奈川：店舗ページ6へ移動しました
2023-04-14 21:42:35,611 [DEBUG] main: 神奈川：店舗ページ6の取得完了
2023-04-14 21:42:38,871 [DEBUG] main: 神奈川：店舗ページ7へ移動しました
2023-04-14 21:42:38,954 [DEBUG] main: 神奈川：店舗ページ7の取得完了
2023-04-14 21:42:41,957 [DEBUG] main: 神奈川：店舗ページ8へ移動しました
2023-04-14 21:42:42,037 [DEBUG] main: 神奈川：店舗ページ8の取得完了
2023-04-14 21:42:44,097 [DEBUG] main: 神奈川：店舗ページ9へ移動しました
2023-04-14 21:42:44,177 [DEBUG] main: 神奈川：店舗ページ9の取得完了
2023-04-14

,店名,電話番号,住所,URL
0,うず潮屋 関内店,050-5869-5207,神奈川県横浜市中区真砂町3-33 セルテ 12F,https://tabelog.com/kanagawa/A1401/A140104/140...
1,ザ・肉餃子 四川厨房 横浜本舗,050-5594-1240,神奈川県横浜市西区南幸2-16-11 二幸ビル 6F,https://tabelog.com/kanagawa/A1401/A140102/140...
2,七輪焼肉 安安 横浜北口店,050-5457-1642,神奈川県横浜市神奈川区鶴屋町2-20-1 ＹＴＵビル Ｂ１Ｆ,https://tabelog.com/kanagawa/A1401/A140101/140...
3,全席個室 楽蔵うたげ 関内駅前店,050-5456-1473,神奈川県横浜市中区港町4-15-2 フィル・パーク 2・3F,https://tabelog.com/kanagawa/A1401/A140104/140...
4,吾照里 ダイナシティ小田原店,050-5872-5981,神奈川県小田原市中里313-12 小田原ダイナシティーウォーク,https://tabelog.com/kanagawa/A1409/A140901/140...
...,...,...,...,...
995,原価ビストロBAN！ 溝の口,044-833-8585,神奈川県川崎市高津区ニ子5-9-6,https://tabelog.com/kanagawa/A1405/A140505/140...
996,まちノ食堂,050-5589-5904,神奈川県横浜市港北区日吉本町1-19-20 厚川ビル 1F,https://tabelog.com/kanagawa/A1401/A140204/140...
997,海鮮カフェ&バー グラベル,050-5597-9614,神奈川県相模原市中央区清新1-6-18 けやきビル 1F,https://tabelog.com/kanagawa/A1407/A140701/140...
998,橙家 横浜みなとみらい東急スクエア店,050-5868-3683,神奈川県横浜市西区みなとみらい2-3-8 みなとみらい東急スクエア③ 4F,https://tabelog.com/kanagawa/A1401/A140103/140...


In [ ]:
# 提出ファイル用出力
df_result.to_excel(SUBMIT_FILE_PATH, header=True, index=False)

# 積み上げリスト保存用出力
df_result.to_csv(ACCU_LIST_PATH, mode='a', header=True, index=False)

In [ ]:
# 積み上げリスト確認
df_accu = pd.read_csv(ACCU_LIST_PATH)
df_accu

,店名,電話番号,住所,URL
0,うず潮屋 関内店,050-5869-5207,神奈川県横浜市中区真砂町3-33 セルテ 12F,https://tabelog.com/kanagawa/A1401/A140104/140...
1,ザ・肉餃子 四川厨房 横浜本舗,050-5594-1240,神奈川県横浜市西区南幸2-16-11 二幸ビル 6F,https://tabelog.com/kanagawa/A1401/A140102/140...
2,七輪焼肉 安安 横浜北口店,050-5457-1642,神奈川県横浜市神奈川区鶴屋町2-20-1 ＹＴＵビル Ｂ１Ｆ,https://tabelog.com/kanagawa/A1401/A140101/140...
3,全席個室 楽蔵うたげ 関内駅前店,050-5456-1473,神奈川県横浜市中区港町4-15-2 フィル・パーク 2・3F,https://tabelog.com/kanagawa/A1401/A140104/140...
4,吾照里 ダイナシティ小田原店,050-5872-5981,神奈川県小田原市中里313-12 小田原ダイナシティーウォーク,https://tabelog.com/kanagawa/A1409/A140901/140...
...,...,...,...,...
995,原価ビストロBAN！ 溝の口,044-833-8585,神奈川県川崎市高津区ニ子5-9-6,https://tabelog.com/kanagawa/A1405/A140505/140...
996,まちノ食堂,050-5589-5904,神奈川県横浜市港北区日吉本町1-19-20 厚川ビル 1F,https://tabelog.com/kanagawa/A1401/A140204/140...
997,海鮮カフェ&バー グラベル,050-5597-9614,神奈川県相模原市中央区清新1-6-18 けやきビル 1F,https://tabelog.com/kanagawa/A1407/A140701/140...
998,橙家 横浜みなとみらい東急スクエア店,050-5868-3683,神奈川県横浜市西区みなとみらい2-3-8 みなとみらい東急スクエア③ 4F,https://tabelog.com/kanagawa/A1401/A140103/140...
